In [1]:
%pip install librosa transformers torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
load_dotenv("key.env")

In [ ]:
#без чанков
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoFeatureExtractor
import torch
import librosa

model = AutoModelForCausalLM.from_pretrained("Vikhrmodels/Borealis", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Borealis")
extractor = AutoFeatureExtractor.from_pretrained("Vikhrmodels/Borealis")

generation_params = {
        "max_new_tokens": 350,
        "do_sample": True,
        "top_p": 0.9,
        "top_k": 50,
        "temperature": 0.2,
 }

model.eval()
model.to("cpu")

waveform, sr = librosa.load("/home/ppwwrr/project_25/voices/voice-2.wav", sr=16_000)

proc = extractor(
        waveform,
        sampling_rate=sr,
        padding="max_length",
        max_length=480_000,
        return_attention_mask=True,
        return_tensors="pt",
    )

mel = proc.input_features.squeeze(0).to("cpu")
att_mask = proc.attention_mask.squeeze(0).to("cpu")

with torch.inference_mode():
    transcript = model.generate(mel=mel, att_mask=att_mask, **generation_params)

print(transcript)



In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#с чанками
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoFeatureExtractor
import torch
import librosa
import numpy as np

# === Настройки ===
model_name = "Vikhrmodels/Borealis"
audio_path = "/home/ppwwrr/project_25/voices/voice-2.wav"
device = "cpu"

# === Загрузка модели и инструментов ===
print("⏳ Загружаем модель...")
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
extractor = AutoFeatureExtractor.from_pretrained(model_name)
model.eval().to(device)

# === Загружаем аудио ===
print("🎧 Загружаем аудио...")
waveform, sr = librosa.load(audio_path, sr=16000)

# === Разбиваем на чанки ===
chunk_duration = 30  # секунд
chunk_size = chunk_duration * sr
chunks = [waveform[i:i + chunk_size] for i in range(0, len(waveform), chunk_size)]
print(f"🔹 Найдено чанков: {len(chunks)}")

# === Обрабатываем каждый чанк ===
transcripts = []

for idx, chunk in enumerate(chunks, start=1):
    print(f"\n🟢 Обработка чанка {idx}/{len(chunks)}...")

    proc = extractor(
        chunk,
        sampling_rate=sr,
        padding="max_length",
        max_length=480_000,
        return_attention_mask=True,
        return_tensors="pt",
    )

    mel = proc.input_features.squeeze(0).to(device)
    att_mask = proc.attention_mask.squeeze(0).to(device)

    # Генерация текста
    with torch.inference_mode():
        output = model.generate(
            mel=mel,
            att_mask=att_mask,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            temperature=0.2,
        )

    # Проверяем, что вернула модель
    if isinstance(output, str):
        text = output.strip()
    elif isinstance(output, torch.Tensor):
        text = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    else:
        text = str(output)

    transcripts.append(text)

# === Склеиваем и выводим результат ===
full_transcript = " ".join(transcripts)

print("\n✅ Транскрипция завершена!")
print("==============================================\n")
print(full_transcript)
print("\n==============================================")


⏳ Загружаем модель...
🎧 Загружаем аудио...
🔹 Найдено чанков: 2

🟢 Обработка чанка 1/2...

🟢 Обработка чанка 2/2...

✅ Транскрипция завершена!

Я смотрела... Короче, у меня вот эти духи появились, чтобы не соврать. В десятом или в одиннадцатом классе? Вот типа вот либо конец десятого, либо начало одиннадцатого. Ну короче, вот они у меня появились. Мне мама заказала, потому что это духи Полиньного отца. Потому что Полина у него никс, она их носила много лет назад. Мне так понравилось. Я такая, блин, что это? Я пошла к ней домой сфоткал и всё — мы их купили. Ещё тогда я смотрела тип на золотом яблоке не стоило типа: «ну девятнадцать», «ну восемнадцать». Ну как бы да, дорого, но нижняя парфюмерия, как бы ладно, ещё можно как бы схавать. Но сорок пять, блять.

